In [1]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    AddChanneld,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    RandAffined,
    Spacingd,
    Invertd,
    Resized,
    ToTensord,
    EnsureTyped,
    EnsureType,

)

# from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import SimpleITK as sitk
import numpy as np
from tqdm.notebook import tqdm, trange


In [5]:
# data_dir = 'E:/Tanvir Mehedi/Tanvir_codes/Data'
data_dir = 'E:/Tanvir Mehedi/Nodule_between_5_10/' 
root_dir =  'E:/Tanvir Mehedi/Tanvir_codes/Saved_Models'

# train_images = sorted(
#     glob.glob(os.path.join(data_dir, "nii_imagelungonly_subset0", "*.nii")))

# train_labels = sorted(
#     glob.glob(os.path.join(data_dir, "nii_spherical_nodule_mask", "*.nii")))

train_images = sorted(
    glob.glob(os.path.join(data_dir, "Only_Lung_Portion", "*.nii")))

train_labels = sorted(
    glob.glob(os.path.join(data_dir, "Spherical_Nodule_Mask", "*.nii")))

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]

train_files, val_files = data_dicts[:16], data_dicts[0:]
print(len(val_files))

11


In [6]:
set_determinism(seed=0)

In [7]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"], a_min=-1200, a_max=600,
            b_min=0.0, b_max=1.0, clip=True),
        Orientationd(keys=["image", "label"], axcodes= 'SPL'),
        Spacingd(keys=["image", "label"], pixdim=(
            1., 1., 1.), mode=("bilinear", "nearest")),
#         Resized(keys=["image"], spatial_size=[256,512,512], mode="trilinear", align_corners = True),
#         Resized(keys=["label"], spatial_size=[256,512,512], mode="nearest"),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(64, 64, 64),
            pos=3,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
#         RandAffined(
#             keys=['image', 'label'],
#             mode=('bilinear', 'nearest'),
#             prob=1.0, spatial_size=(64, 64, 64),
#             rotate_range=(0, 0, np.pi/3),
#             scale_range=(0.1, 0.1, 0.1)),
        
#         ToTensord(keys=["image", "label"]),
        EnsureTyped(keys=["image", "label"]),
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"], a_min=-1200, a_max=600,
            b_min=0.0, b_max=1.0, clip=True),
        Orientationd(keys=["image", "label"], axcodes="SPL"),
        Spacingd(keys=["image", "label"], pixdim=(
            1., 1., 1.), mode=("bilinear", "nearest")),
#         Resized(keys=["image"], spatial_size=[256,512,512], mode="trilinear", align_corners = True),
#         Resized(keys=["label"], spatial_size=[256,512,512], mode="nearest"),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        
        ToTensord(keys=["image", "label"]),
    ]
)

In [17]:
check_ds = Dataset(data=train_files, transform=train_transforms)
#check_ds = Dataset(data=train_files)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
image, label = (check_data["image"], check_data["label"])
print(f"image shape: {image.shape}, label shape: {label.shape}")


image shape: torch.Size([4, 1, 64, 64, 64]), label shape: torch.Size([4, 1, 64, 64, 64])


In [ ]:
torch.any(label[label==1])

In [ ]:
torch.max

In [ ]:
sli = image[0,0,32,:,:]
plt.hist(sli)
plt.show()

In [1]:
a=4
for j in range(a):
    print("Number of Sample:",j)
    plt.figure("check", (12, 6))
    for i in range(image.shape[2]):
        print('image', i)
        plt.subplot(1, 2, 1)
        plt.title("image")
        plt.imshow(image[j,0,i, :, :], cmap="gray")
        plt.subplot(1, 2, 2)
        plt.title("label")
        plt.imshow(label[j,0,i, :, :],  cmap="gray")
        plt.show()

Number of Sample: 0


NameError: name 'plt' is not defined

In [9]:
# Model Parameters
batch_size = 1
num_workers = 2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [10]:
# Train
train_ds = Dataset(data = train_files, transform = train_transforms)
train_loader = DataLoader(train_ds, batch_size = batch_size, shuffle = True, num_workers = num_workers)

# Validation
val_ds = Dataset(data = val_files, transform = val_transforms)
val_loader = DataLoader(val_ds, batch_size = 1, num_workers = num_workers)

# Test
# test_ds = Dataset(data = test_files, transform = test_transforms)
# test_loader = DataLoader(test_ds, batch_size = batch_size, num_workers = num_workers)


In [ ]:
label.shape

In [12]:
len(train_loader)

4

In [32]:
count = 0

for a,b in enumerate(train_loader) :
    
    img = b['image']
    msk = b['label']
    
    for i in range(16):
        
        label = msk[i,:,:,:,:]
    #         print(label.shape)

        if torch.any(label[label==1]) :

            count = count+1
    print(count)


13
24
35
49


In [14]:
for i in range(16):
        
    label = msk[i,:,:,:,:]
#         print(label.shape)

    if torch.any(label[label==1]) :

        count = count+1
print(count)

1
2
3
4
5
6
7
8


In [ ]:
torch.any(msk[msk==1])

In [11]:
model = UNet(
    spatial_dims = 3,
    in_channels = 1,
    out_channels = 2,
#     channels = (32, 64, 128, 256, 512),
    channels=(16, 32, 64, 128, 256),
    strides = (2, 2, 2, 2),
    num_res_units = 2,
    norm = Norm.BATCH,
).to(device)


In [12]:
# loss_function = DiceLoss(to_onehot_y = False, sigmoid = True)
loss_function = DiceLoss(to_onehot_y=True, softmax=True)

optimizer = torch.optim.Adam(model.parameters(), 0.1)

scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01,step_size_up=5,
                                              mode="triangular2", cycle_momentum = False)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.1)

dice_metric = DiceMetric(include_background = False, reduction = "mean")


In [ ]:
def save_checkpoint(state, filename = 'nodule_seg_3d_run_monai.pt.tar'):
    
    print('=> Saving Checkpoint')
    torch.save(state, filename)
    

In [13]:
max_epochs = 250
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
post_pred = Compose([EnsureType(), AsDiscrete(argmax=True, to_onehot=True, num_classes=2)])
post_label = Compose([EnsureType(), AsDiscrete(to_onehot=True, num_classes=2)])

In [ ]:
for epoch in range(max_epochs):
    
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    
    model.train()
    epoch_loss = 0
    step = 0
    
    for batch_data in train_loader:
        step += 1
        inputs, labels = (
            batch_data["image"].to(device),
            batch_data["label"].to(device),
        )
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 2. , norm_type=2.0)
        optimizer.step()
        epoch_loss += loss.item()
        print(
            f"{step}/{len(train_ds) // train_loader.batch_size}, "
            f"train_loss: {loss.item():.4f}")
    
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        
        model.eval()
        
        with torch.no_grad():
            
            for val_data in val_loader:
                val_inputs, val_labels = (
                    val_data["image"].to(device),
                    val_data["label"].to(device),
                )
# -----------
                # compute metric for current iteration  
#                 val_outputs = model(val_inputs)
#                 val_outputs = torch.sigmoid(val_outputs)>0.5
# ----------
                roi_size = (64, 64, 64)
                sw_batch_size = 4
                val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
                val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                val_labels = [post_label(i) for i in decollate_batch(val_labels)]
                dice_metric(y_pred = val_outputs, y = val_labels)
                

            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()

            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                
                checkpoint = {'model_state' : model.state_dict(), 'optimizer_state' : optimizer.state_dict(), 
                              'Dice_Score' : metric } 
                save_checkpoint(checkpoint)
                
                torch.save(model.state_dict(), os.path.join(
                    root_dir, "best_metric_model.pth"))
                print("saved new best metric model")
            print(
                f"current epoch: {epoch + 1} current mean dice: {metric:.4f}"
                f"\nbest mean dice: {best_metric:.4f} "
                f"at epoch: {best_metric_epoch}"
            )
    scheduler.step()  # decay LR (if step_size hit)
    curr_lr = scheduler.get_last_lr()
    print('Current learning rate', curr_lr)

In [ ]:
print(curr_lr)

In [15]:
path = 'E:/Tanvir Mehedi/Tanvir_codes/Saved_Models/best_metric_model.pth'
model.load_state_dict(torch.load(path))


<All keys matched successfully>

In [16]:
model.eval()

with torch.no_grad():

    for val_data in val_loader:
        val_inputs, val_labels = (
            val_data["image"].to(device),
            val_data["label"].to(device),
        )
# -----------
        # compute metric for current iteration  
#                 val_outputs = model(val_inputs)
#                 val_outputs = torch.sigmoid(val_outputs)>0.5
# ----------
        roi_size = (64, 64, 64)
        sw_batch_size = 1
        val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)

        val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
#         val_labels = [post_label(i) for i in decollate_batch(val_labels)]
        
        dice_metric(y_pred = val_outputs, y = val_labels)


    # aggregate the final mean dice result
    metric = dice_metric.aggregate().item()
    
    print(metric)


0.003050349187105894


In [ ]:
val_labels[0].shape

In [ ]:
val_outputs[0].shape

In [ ]:
a = val_labels[0].argmax(0).cpu()
a.shape

In [ ]:
import matplotlib.pyplot as plt

imgs = a

for i in range(len(imgs)):
    print("image %d" % i)
#     fig,ax = plt.plot(figsize=[8,8])
    plt.figure(figsize=(8, 8))
    plt.imshow(imgs[i],cmap='gray')
    plt.show()
imgs.shape

In [ ]:
print(inputs.dtype), print(labels.dtype), print(outputs.dtype)

In [ ]:
val_labels[0].shape

In [ ]:
plt.imshow(val_labels[0][1,150,:,:].detach().cpu().numpy(), cmap = plt.cm.bone)

In [ ]:
plt.imshow(torch.sigmoid(outputs[2,0,50,:,:]).detach().cpu().numpy(), cmap = plt.cm.bone)

In [ ]:
import monai; monai.config.print_config()

In [ ]:
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"], a_min=-1200, a_max=600,
            b_min=0.0, b_max=1.0, clip=True),
        Orientationd(keys=["image", "label"], axcodes="SPL"),
        Spacingd(keys=["image", "label"], pixdim=(
            1., 1., 1.), mode=("bilinear", "nearest")),
        Resized(keys=["image"], spatial_size=[256,512,512], mode="trilinear", align_corners = True),
        Resized(keys=["label"], spatial_size=[256,512,512], mode="nearest"),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(64, 64, 64),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        ToTensord(keys=["image", "label"]),
    ]
)

In [ ]:
val_ds = Dataset(data = val_files, transform = val_transforms)
val_loader = DataLoader(val_ds, batch_size = 1, num_workers = num_workers)

In [ ]:
model.eval()
        
with torch.no_grad():

    for val_data in val_loader:
        val_inputs, val_labels = (
            val_data["image"].to(device),
            val_data["label"].to(device),
        )
        val_outputs = model(val_inputs)
        loss = loss_function(val_outputs, val_labels)
        print(loss)

In [ ]:
a = val_outputs.argmax(1).cpu()
a.shape

In [ ]:
val_outputs.shape